In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv
import sleap

#from visualize import Visualize
#from track import Track


Autosaving every 180 seconds


In [ ]:
import cv2
from tqdm import tqdm, trange


def make_video_centroid(
                    tracks_huddles,
                    tracks_features,
                    tracks_huddle_ethogram,
                    fname_video,
                    fname_video_out,
                    start,
                    end,
                    fps,
                    shrink,):
    # colors:
    colors = [
      (0,0,255),
      (255,0,0),
      (0,255,255),
      (139,0,139),
      (0,255,0),
      (255,255,0),
      (125,125,0),
    ]
    #
    names = ['female', 'male', 'pup1', 'pup2', 'pup3','pup4']
    clrs = ['royalblue', 'firebrick', 'goldenrod', 'seagreen', 'rebeccapurple','lightskyblue']
    if start is None:
        start = 0
    if end is None:
        end = tracks.shape[0]
    # load and videos
    video_name = fname_video
    fname_out = (video_name[:-4] + '_' + str(start) + "_" + str(end) + '_'+ fname_video_out+'.mp4')
    if False:  # os.path.exists(fname_out):
        print("fname: exists", fname_out)
        return

    # rescale tracks
    tracks= tracks_huddles/shrink
    # load original vid
    original_vid = cv2.VideoCapture(video_name)
    width = original_vid.get(cv2.CAP_PROP_FRAME_WIDTH )
    height = original_vid.get(cv2.CAP_PROP_FRAME_HEIGHT )
    fps_out =  fps
    print ("width, height: ", width,height)
    # video settings
    size_vid = np.int32(np.array([width, height])/shrink)
    dot_size = int(12/shrink)
    thickness = -1
    #print ("size vid: ", size_vid)
    # make new video
    fourcc = cv2.VideoWriter_fourcc('M', 'P', 'E', 'G')
    video_out = cv2.VideoWriter(fname_out, fourcc, fps_out, (size_vid[0], size_vid[1]), True)
    # set frames to new ones
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
    font = cv2.FONT_HERSHEY_PLAIN

    #
    for n in trange(start, end, 1):
        #
        ret, frame = original_vid.read()
        # scale frame
        shrink_ratio = 1/shrink
        frame = cv2.resize(frame, # original image
                  (0,0), # set fx and fy, not the final size
                  fx=shrink_ratio,
                  fy=shrink_ratio,
                  interpolation=cv2.INTER_NEAREST)
        
        # plot huddles
        #if False:
        for i in range(tracks_huddles.shape[1]):
            color = colors[6]
            # for f in range(tracks.shape[2]):
            x = tracks_huddles[n, i, 0]
            y = tracks_huddles[n, i, 1]
            if np.isnan(x):
                continue
            center_coordinates_huddle = (int(x), int(y))
            radius = int(dot_size/2)
            ######## IF HUDDLE #######
            radius = 100
            if False:
                frame = cv2.circle(frame,
                                center_coordinates_huddle,
                                radius,
                                color,
                                thickness)

                    
        # plot huddle ethogram markers
        x = [-1,0,1,-1,0,1]
        y = [0,0,0,1,1,1]
        for i in range(tracks_huddle_ethogram.shape[1]):
            color = colors[i]
            # for f in range(tracks.shape[2]):
            idx = tracks_huddle_ethogram[n//fps, i]
            if idx==0:
                continue
            #print (center_coordinates_huddle[0]+int(x)*5)
            center_coordinates = (center_coordinates_huddle[0]+int(x[i])*75, 
                                  center_coordinates_huddle[1] + int(y[i])*75)
            radius = int(dot_size/2)
            ######## IF HUDDLE #######
            radius = 30
            frame = cv2.circle(frame,
                                center_coordinates,
                                radius,
                                color,
                                thickness)
            
                    
        # plot tracks
        for i in range(tracks_features.shape[1]):
            color = colors[i]
            # for f in range(tracks.shape[2]):
            x = tracks_features[n, i, 0]
            y = tracks_features[n, i, 1]
            if np.isnan(x):
                continue
            center_coordinates = (int(x), int(y))
            radius = int(dot_size/2)
            ######## IF HUDDLE #######
            radius = 12
            frame = cv2.circle(frame,
                                center_coordinates,
                                radius,
                                color,
                                thickness)
        #
        video_out.write(frame)
    video_out.release()

#
shrink = 1 # shrink image factor
fname_video_out = 'final'
fname_video = '/home/cat/Downloads/2020_08_01_07_21_02_430892_compressed_defished_shrink_cropped.mp4'
#features_huddles = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/huddles/2020_08_01_20_04_11_490071_compressed_Both_huddle_spine_fixed_interpolated.npy')
#features_tracks = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/features/2020_08_01_20_04_11_490071_compressed_Both_spine_nohuddle.npy')
features_tracks = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/features/2020_08_01_07_21_02_430892_compressed_Night_spine_nohuddle.npy')
features_huddles = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/huddles/2020_08_01_07_21_02_430892_compressed_Night_huddle_spine_fixed_interpolated.npy')
features_huddle_ethogram = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/features/2020_08_01_07_21_02_430892_compressed_Night_spine_nohuddle_huddle_ethogram.npy').T
#
####################################################
########## MAKE MOVIES OF SPINE CENTRES ############
####################################################
start = 0
end = 26000
fps = 24
scale = 1

#
make_video_centroid(features_huddles, 
                    features_tracks, 
                    features_huddle_ethogram,
                    fname_video, 
                    fname_video_out, 
                    start, 
                    end, 
                    fps, 
                    scale)
                    
  

OpenCV: FFMPEG: tag 0x4745504d/'MPEG' is not supported with codec id 2 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


width, height:  900.0 700.0


  1%|█                                                                                      | 304/26000 [00:04<05:40, 75.36it/s]

In [13]:
features_huddle_ethogram = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/features/2020_08_01_07_21_02_430892_compressed_Night_spine_nohuddle_huddle_ethogram.npy')



In [14]:
print (features_huddle_ethogram.shape)

(6, 1201)
